In [60]:
import json
import re
import os
import ast
import csv
import pandas as pd

In [61]:
def decode_utf8(item):
    if isinstance(item, str):  # If item is a string
        try:
            return item.encode('latin1').decode('utf-8')
        except:
            return item  # Return original if decoding fails
    elif isinstance(item, list):  # If item is a list
        return [decode_utf8(x) for x in item]
    elif isinstance(item, dict):  # If item is a dictionary
        return {key: decode_utf8(value) for key, value in item.items()}
    else:  # If item is neither string, list nor dict (could be int, bool, etc.)
        return item

In [62]:
def clean_n_decode(data_in, file_name):
    splited_data = data_in.replace("b'{", 'b"{').split('b"')
    arr_data = [spld.replace("\\'", "'") for spld in splited_data]

    json_data_arr = []
    count_try, count_success = 0,0
    
    for i in range(len(arr_data)):
        if i == 0:
            continue
        elif "TikTokApi.video(" in arr_data[i]:
            continue
        else:
            try:
                count_try += 1
                # clean & decode UTF-8
                content = arr_data[i].replace("'", '"')[:-2]
                content = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", content) # reg replace from Don"t to Don't

                # # analyze error
                # pattern1 = r'"title":\s*"(.*?)"'
                # pattern2 = r'"desc":\s*"(.*?)"'
                # updated_string = re.sub(pattern1, '"title": "ignore string error"', content)
                # updated_string = re.sub(pattern2, '"desc": "ignore string error"', updated_string).replace("True", "true").replace("False", "false")

                str_data = ast.literal_eval(content)
                decoded_data = str(decode_utf8(str_data)).replace("'", '"').replace("True", "true").replace("False", "false").replace("\\\\n", " ")
                updated_string = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", decoded_data) # reg replace from Don"t to Don't
                # print(updated_string) # print and paste this on the .json file to see if there's an error

                # turn string into object
                json_data = json.loads(updated_string)
                id = json_data["id"]

                # print(id, json_data)

                # # export to check json data
                # file_path = r"./exported-accountpage-json-error-check/" + file_name[:-4] + "-" + id + ".json"
                # with open(file_path, "w", encoding='utf-8') as file:
                #     json.dump(json_data, file, indent=4, ensure_ascii=False)
                
                
                json_data_arr.append(json_data)

                count_success += 1
                # break # uncomment this for one time loop
            except:
                print(file_name + ' error inside clean_n_decode() function')
                # break
    print(count_success, "/",count_try)
    return json_data_arr


In [63]:
def extract_nested_keys(my_dict):

    keys_to_extract = [
        ["author", "uniqueId"],
        ["author", "signature"],
        ["author", "verified"],
        "desc",
        "verified",
        "createTime",
        "digged",
        "duetDisplay",
        "duetEnabled",
        "forFriend",
        "id",
        "itemCommentStatus",
        ["music", "authorName"],
        ["music", "coverLarge"],
        ["music", "duration"],
        ["music", "id"],
        ["music", "original"],
        ["music", "playUrl"],
        ["music", "title"],
    ]

    result = {}
    for key in keys_to_extract:
        if isinstance(key, list):
            nested_result = my_dict
            for nested_key in key:
                if nested_key in nested_result:
                    nested_result = nested_result[nested_key]
                    # print(nested_key)
                else:
                    nested_result = None
                    break
            if nested_result is not None:
                result[str("_".join(key))] = nested_result
        elif key in my_dict:
            result[key] = my_dict[key]
    return result



In [64]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

    # print(hashtag_arr)
    joined_string = ', '.join(hashtag_arr)
    return joined_string

In [65]:
def extract_data(json_data):
    try:
        extracted_values = extract_nested_keys(json_data)

        if json_data["contents"] is None:
            return extracted_values
        else:
            extracted_values["hashtagNames"] = extract_hashtags(json_data["contents"])
            return extracted_values
    except:
        # print("return without contents -> hashtagNames fields")
        return extracted_values


In [66]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

    # print(hashtag_arr)
    # joined_string = ', '.join(hashtag_arr)
    return hashtag_arr

In [67]:
def clear_csv_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

In [68]:
origin_folder_path = r"./txt-accounts-files"
csv_file_path = "AccountPages.csv"
extracted_df = None

clear_csv_file(csv_file_path)

for i, file_name in enumerate(os.listdir(origin_folder_path)):
    if file_name.endswith('.txt'):
        file_path = os.path.join(origin_folder_path, file_name)
        print("\n", i+1, file_name)

        # try:
        with open(file_path, "r") as file:
            file_data = file.read()
        
        # clean and decode -> extract columns
        json_data_arr = clean_n_decode(file_data, file_name)
        extracted_data_json_list = [extract_data(json_data) for json_data in json_data_arr] 
        
        # create dataframe
        if extracted_df is None:
            extracted_df = pd.DataFrame(columns=extracted_data_json_list[0].keys())

        filtered_list = [data for data in extracted_data_json_list if data is not None] # one or more elements in your extracted_data_json_list is None instead of a dictionary
        df = pd.DataFrame.from_records(filtered_list)
        extracted_df = pd.concat([extracted_df, df], ignore_index=True)

        # except:
        #     print(file_name + ' error')

extracted_df.to_csv(csv_file_path, mode='a', header=True, index=False)


 1 1997thantzinoo1.txt
1997thantzinoo1.txt error inside clean_n_decode() function
34 / 35

 2 alocatagram.txt
35 / 35

 3 bikesure2.txt
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt error inside clean_n_decode() function
bikesure2.txt e